# FAST-UAV - Tutorial
*Author: Félix Pollet - 2023* <br>

Welcome to FAST-UAV, a framework for the multidisciplinary design of electric UAVs.<br>
FAST-UAV is based on the open source library [FAST-OAD](https://fast-oad.readthedocs.io), which provides numerical capabilities for aircraft design. The computational core of FAST-OAD is itself based on the [OpenMDAO framework](https://openmdao.org/).

This notebook aims to introduce the basic usage of FAST-UAV. By the end of this tutorial, you will be able to:
1. Set up a UAV design problem in FAST-UAV
2. Run an optimization to obtain the best feasible design for a given set of specifications
3. Visualize the results

Note that the way FAST-UAV work is very similar to FAST-OAD. Don't hesitate to have a look at the [FAST-OAD documentation](https://fast-oad.readthedocs.io) to learn more about the basics.

## 1. Setting up and analyzing the initial problem

### a) Folders and files

To organize our work, we will use two different folders, namely `data/` and `workdir/`. 

The `data/` folder stores the configuration files, the mission files and the source (reference) files. The `workdir/` folder stores temporary files, for instance copies of the problem inputs and outputs. Let explain in more details the different files:

* A `configuration.yaml` file is a minimalistic interface where you can define the design problem, including the UAV concept (multirotor, fixed wing, or hybrid) to be optimized or evaluated, the optimization variables/constraints/objectives, and the optimization algorithm settings. Default configuration files are provided by FAST-UAV for the main UAV concepts and design problems, which you can find in the `data/configurations` folder.

* The design problem definition requires a `missions.yaml` file. This file allows you to define the mission profile, e.g. if you have a climb phase followed by a cruise and a hover. This definition is purely qualitative here. The values associated to each of the flight segments (e.g. climb rate and hover duration) are provided in the `problem_inputs.xml` file described hereafter.

* The input file `problem_inputs.xml` allows you to specify the values for the design specifications (e.g. mission requirements and number of propellers) and, for advanced users, to access and fine-tune the model parameters. In practice, a small interface is provided (see further below in this notebook) to access and modify the input file rather than having to open/modify the raw XML file. As a starting point, FAST-UAV provides some examples of input files, which can be found in the `data/source_files` directory.

### b) Case study
In this tutorial, the goal is to find the optimal sizing of a hexacopter UAV given a set of user-defined specifications. As an illustrative case, the performance specifications for the multirotor UAV we aim to optimize are similar to those of the [DJI Matrice 600 Pro](https://www.dji.com/matrice600-pro): design payload of 5.5 kg, 18 minutes hover endurance, and thrust-to-weight ratio of 1.97, among others.

In the following cells, the required librairies are imported and the paths to the folders and files are declared.

In [ ]:
# Import required librairies
import os.path as pth
import sys
import logging
import fastoad.api as oad
import shutil
from fastuav.utils.postprocessing.analysis_and_plots import multirotor_geometry_plot, mass_breakdown_sun_plot_drone

sys.path.append(pth.abspath("."))

#logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

# For using all screen width
from IPython.display import display, HTML, IFrame
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
# Paths to folders and files
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"
CONFIGURATION_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "configurations")
SOURCE_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "source_files")

CONFIGURATION_FILE = pth.join(CONFIGURATION_FOLDER_PATH, "multirotor_mdo.yaml")
SOURCE_FILE = pth.join(SOURCE_FOLDER_PATH, "problem_inputs_DJI_M600.xml")

A useful feature is the [N2 diagram](https://openmdao.org/newdocs/versions/latest/features/model_visualization/n2_basics/n2_basics.html) visualization, which shows the structure and links between the models involved in the design process:

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
IFrame(src=N2_FILE, width="100%", height="500px")

Now, let's have a look at the inputs of the design problem. First, we generate the input file for the problem as a copy of the source (reference) file for the DJI M600 specifications (`problem_inputs_DJI_M600.xml`):

In [ ]:
input_file = oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

You can now checkout the generated [input file](./workdir/problem_inputs.xml). The values in this file can be modified by the user and will be considered by FAST-UAV when executing the computational process.<br>
The `variable-viewer` provides a way to inspect and modify the content of the inputs file. The dropdown lists above the table allow to filter the displayed variable.

The variables are grouped into four main categories:
* The *'mission'* category contains all variables related to the sizing mission (and to any other off-design mission, as we will see in notebook [1b_Mission_performance](1b_Mission_performance.ipynb)), for example the design payload mass and the hover endurance requirement.
* The *'models'* category relates to the parameters of the models supporting the design process. Fine-tuning of these parameters is recommended only for advanced users that are familiar with scaling laws and regression models.
* The *'optimization'* category contains any variable related to the optimization process, i.e. the design variables that will be varied by the optimizer and the constraints.
* Finally, the *'data'* category contains all remaining variables that relate to the components characteristics and performance, for example the propellers diameter and rotation speeds in various operating conditions.

Feel free to modify the values (e.g., the payload mass requirement specified by the variable *"mission:sizing:payload:mass"*), and make sure to click on the Save button before going any further.

In [ ]:
oad.variable_viewer(input_file)

## 2. Running the design optimization

You can now run the optimization process. In the default [configuration file](./data/configurations/multirotor_mdo.yaml), a mass minimization objective is defined. That is, we seek the lightest design that meets the requirements (e.g. hover endurance of 18 min). 

In [ ]:
optim_problem = oad.optimize_problem(CONFIGURATION_FILE, overwrite=True)

If everything went well, you should see "Optimization Complete" displayed in the output of previous cell. Let's save the optimization results permanently in the `data/` folder by copying the temporary output file created by FAST-UAV:

In [ ]:
output_file = optim_problem.output_file_path
shutil.copy(output_file, 
            pth.join(SOURCE_FOLDER_PATH, 'problem_outputs_DJI_M600_mdo.xml')
)

*Note: Advanced users may replace the optimization objective, for example by a range/endurance maximization objective. To do so, you have to modify the last lines of the configuration file related to the optimization objective:*

```yaml
optimization:
  objective:
    #- name: data:weights:mtow  # mass minimization objective
    
    - name: data:performance:range:cruise  # range maximization objective
      scaler: -1  # this multiplier defines a maximization objective instead of a minimization 
```

*An example of optimization with a range maximization objective is provided in notebook [2_FixedWing_Design](2_FixedWing_Design.ipynb).*

## 3. Results and analysis
There are several ways of analyzing the results of the optimization process.

First, the `optimizer_viewer` provides a convenient summary of the optimization variables, constraints, and objective function at convergence. If design variables or constraints have active bounds, they are highlighted in yellow, and if they are violated, they are highlighted in red.

In [ ]:
oad.optimization_viewer(CONFIGURATION_FILE)

Secondly, you can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [ ]:
oad.variable_viewer(output_file)

Finally, a few functions are provided to plot the geometry of the optimal design and its mass breakdown:

In [ ]:
fig = multirotor_geometry_plot(output_file)
fig.show()

In [ ]:
fig = mass_breakdown_sun_plot_drone(output_file)
fig.show()